<a href="https://colab.research.google.com/github/Uditxarma/NextWordPrediction_Project/blob/main/NextWordPrediction_Load%26Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file = open("/content/drive/MyDrive/Next Word Prediction/DataWork.txt","r",encoding ="utf8")  

In [6]:
lines=[] 
for i in file:
  lines.append(i)
data="" 
for i in lines:
  data=' '.join(lines) 
  data=data.replace('\n','').replace('\r','').replace('\ufeff','').replace('"','').replace('.','').replace(',','').replace('!','').replace('@','').replace('#','').replace('$','').replace('%','').replace('^','').replace('&','').replace('*','').replace('(','').replace(')','').replace('+','').replace('/','').replace('<','').replace('>','').replace('?','').replace('{','').replace('}','').replace('[','').replace(']','').replace(';','').replace(':','').replace('|','')
data=data.split() 
data=' '.join(data) 
data[:500]


'The Project Gutenberg eBook of Persuasion by Jane Austen This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever You may copy it give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at wwwgutenbergorg If you are not located in the United States you will have to check the laws of the country where you are located before using this eBook Title P'

In [7]:
len(data)

469882

In [8]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([data])
pickle.dump(tokenizer,open('token.pk2','wb'))
sequence_data = tokenizer.texts_to_sequences([data])[0] 
sequence_data[:15]

[1, 134, 136, 706, 4, 807, 29, 2123, 2124, 45, 706, 35, 18, 1, 267]

In [9]:
len(sequence_data)

86730

In [10]:
vocab_size = len(tokenizer.word_index)+1 
print(vocab_size)

6888


In [11]:
sequences = [] 
for i in range(3,len(sequence_data)):
   words = sequence_data[i-3:i+1]     
   sequences.append(words)
print("The Length of sequence are: ",len(sequences)) 
sequences =np.array(sequences)
sequences[:10]

The Length of sequence are:  86727


array([[   1,  134,  136,  706],
       [ 134,  136,  706,    4],
       [ 136,  706,    4,  807],
       [ 706,    4,  807,   29],
       [   4,  807,   29, 2123],
       [ 807,   29, 2123, 2124],
       [  29, 2123, 2124,   45],
       [2123, 2124,   45,  706],
       [2124,   45,  706,   35],
       [  45,  706,   35,   18]])

In [12]:
X=[] 
Y=[] 
for i in sequences:     
  X.append(i[0:3])
  Y.append(i[3])
X=np.array(X) 
Y=np.array(Y)
print("Data: ",X[:10]) 
print("Response: ",Y[:10])

Data:  [[   1  134  136]
 [ 134  136  706]
 [ 136  706    4]
 [ 706    4  807]
 [   4  807   29]
 [ 807   29 2123]
 [  29 2123 2124]
 [2123 2124   45]
 [2124   45  706]
 [  45  706   35]]
Response:  [ 706    4  807   29 2123 2124   45  706   35   18]


In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3)) 
model.add(LSTM(1000,return_sequences=True)) 
model.add(LSTM(1000)) 
model.add(Dense(1000,activation='relu')) 
model.add(Dense(vocab_size, activation ='softmax'))

In [15]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Next Word Prediction/next_word.h5')
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             68880     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 6888)              6894888   
                                                                 
Total params: 20,012,768
Trainable params: 20,012,768
Non-trainable params: 0
_________________________________________________________________


In [18]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('/content/drive/MyDrive/Next Word Prediction/next_word.h5')
tokenizer = pickle.load(open('token.pk2', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [19]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue
        

Enter your line: He considered the
['He', 'considered', 'the']
1/1 [==============================] - 2s 2s/step
blessing
Enter your line: had attempted no
['had', 'attempted', 'no']
1/1 [==============================] - 0s 42ms/step
apology
Enter your line: 0
Execution completed.....
